# Лабораторная работа 3. Исследование оптических свойств фотонных кристаллов

**Цель** – промоделировать и экспериментально исследовать оптические свойства одномерных фотонных кристаллов.

Так как основные сведения, касающиеся технологии синтеза и свойств фотонных кристаллов, изложены на Stepik, не будем здесь их дублировать. Минимальные необходимые для понимания определения даются по ходу описания лабораторной работы. 

Перед началом работы введите номер варианта, запустив ячейку ниже. Для студентов СПбГЭТУ "ЛЭТИ" номер варианта доступен по ссылке на файл в соответствующей беседе vk, там же, где и варианты для первой лабораторной работы (можно найти в беседе по слову "вариант"). Остальные могут использовать вариант 0.

In [ ]:
var = int(input("Введите номер варианта: "))

## 1. Метод матриц переноса

В качестве математического аппарата при расчете и анализе многослойных тонкопленочных структур используется метод матриц переноса (МП). Распространение электромагнитного излучения в методе МП задается вектором, описывающим движение света в каждой точке многослойной структуры через две волны: распространяющуюся в том же направлении, что исходный луч — $E^R$ и в обратном направлении — $E^L$:

$$E = 
\begin{pmatrix}
  E^R\\
  E^L
\end{pmatrix}$$

Учесть распространение света сквозь среду и через границы двух сред можно при помощи матриц двух типов: матрица $A$ связывает вектора слева и справа от границы раздела, матрица $B$ описывает распространение волны в однородной среде между границами раздела:

$$
A =
\dfrac{1}{2n_2}
\begin{pmatrix}
  n_1+n_2& n_2-n_1\\
  n_2-n_1& n_1+n_2
\end{pmatrix},
$$

$$
B =
\begin{pmatrix}
  \exp\left(\dfrac{2\pi i n L}{\lambda}\right)& 0\\
  0& \exp\left(-\dfrac{2\pi i n L}{\lambda}\right)
\end{pmatrix},
$$

где $n_1$, $n_2$ — показатели преломления чередующихся слоев, $n$, $L$ — показатель преломления и толщина отдельного слоя, $\lambda$ — длина волны.

Взаимодействие света с какой-либо структурой описывает матрицей переноса $M$, являющейся перемножением матриц $A$ и $B$ в соответствии с чередованием границ и слоев. Свет исходно распространяется из какой-либо среды, обычно воздуха, что также нужно учитывать в расчете, помня о границах раздела между многослойной структурой и средой, в которую она  помещена. Так, в простейшем случае для свободной тонкой пленки толщиной $L$ матрица $M$ представляет собой следующее перемножение матриц:

$$M = A_1 B_1 A_2,$$

где $A_1$, $A_2$ — соответственно матрицы $A$ для границ воздух/тонкая пленка и тонкая пленка/воздух, а матрица $B_1$ — матрица $B$ для материала тонкой пленки толщиной $L$.  Соответственно для многослойной системы и, в частности, фотонного кристалла, матрица переноса $M$ задается перемножением всех соответствующих матриц $A$ и $B$.

Матрица переноса $M$ является решением матричного уравнения следующего вида:

$$
\begin{pmatrix}
  t\\
  0
\end{pmatrix} = 
M \cdot 
\begin{pmatrix}
  1\\
  r
\end{pmatrix}$$

$$
\begin{pmatrix}
  t\\
  0
\end{pmatrix} = 
\begin{pmatrix}
  a&b\\
  c&d
\end{pmatrix} \cdot 
\begin{pmatrix}
  1\\
  r
\end{pmatrix},$$

где параметры $t$ и $r$ характеризуют изменение напряженности распространяющейся электромагнитной волны (прошедшую и отраженую компоненты). Решая матричное уравнение, получаем:

$$
r = -c/d,$$

$$t = a+br.
$$

Учитывая, что интенсивность излучения определяется квадратом напряженности поля, получаем

$$R = |c/d|^2,$$
$$T = 1-R.$$

Обратите внимание, что исходно $t$ и $r$ в общем случае имеют комплексные значения. То есть $R$ (коэффициент отражения) представляет собой квадрат модуля комплексного числа.

Ниже приведен программный код, реализующий расчет оптических спектров фотонных кристаллов методом матриц переноса. Запустите ячейку, чтобы можно было использовать функции из этого программного блока.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

indices = {'air': '1*x/x',
           'SiO2': 'np.sqrt(1+0.6961663/(1-np.power(0.0684043/x, 2))+0.4079426/(1-np.power(0.116241/x,2))+0.8974794/(1-np.power(9.896161/x,2)))',
           'TiO2': 'np.sqrt(5.913+0.2441/(np.power(x, 2)-0.0803))'}

h = [0, 1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7]

def n(material, x):
    nL = lambda x: eval(indices[material])
    return nL(x)


def dot_product(M, B):
    return np.array([[M[0,0,:]*B[0,0,:]+M[0,1,:]*B[1,0,:],
                   M[0,0,:]*B[0,1,:]+M[0,1,:]*B[1,1,:]],
                  [M[1,0,:]*B[0,0,:]+M[1,1,:]*B[1,0,:],
                   M[1,0,:]*B[0,1,:]+M[1,1,:]*B[1,1,:]]])


def A(wavelengths, material_1, material_2):
    n1 = n(material_1, wavelengths)
    n2 = n(material_2, wavelengths)
    return [[n2+n1, n2-n1],
            [n2-n1, n2+n1]]/(2*n2)


def B(wavelengths, material, L):
    nm = n(material, wavelengths)
    return np.array([[np.exp(2*np.pi*1j*nm*L/wavelengths),
                      np.zeros(nm.size)],
                     [np.zeros(nm.size),
                      np.exp(-2*np.pi*1j*nm*L/wavelengths)]])


def M(structure, wavelengths):
    materials = structure[0::2]
    thicknesses = np.array(structure[1::2])/1000
    matrix_M = A(wavelengths, materials[0], materials[1])
    for i in range(len(materials)-2):
        matrix_B = B(wavelengths, materials[i+1], thicknesses[i+1])
        matrix_M = dot_product(matrix_M, matrix_B)
        matrix_A = A(wavelengths, materials[i+1], materials[i+2])
        matrix_M = dot_product(matrix_M, matrix_A)
    return matrix_M


def T(structure, xmin=300, xmax=1000):
    wavelengths = np.linspace(xmin/1000, xmax/1000, num=1000)
    M_matrix = M(structure, wavelengths)
    T = 1 - np.abs(-M_matrix[1, 0]/M_matrix[1, 1])**2
    plt.xlabel(r'$\lambda$, нм')
    plt.ylabel(r'$T$, %')
    plt.axis([xmin, xmax, 0, 100])
    plt.plot(wavelengths*1000, T*100, color='blue')
    plt.tight_layout()
    
print('Теперь доступны все необходимые функции.')

##  2. Моделирование спектров пропускания фотонных кристаллов
### 2.1. Исследование свойств материалов оптической ячейки

Под оптической ячейкой фотонного кристалла понимается пара слоев заданной толщины, которые образуют периодическую структуру фотонного кристалла. Материалами оптической ячейки в данной работе являются диоксид кремия $\text{SiO$_2$}$ и диоксид титана $\text{TiO$_2$}$. О материале с меньшим значением показателя преломления говорят как об оптически менее плотном. В данном случае таким материалом является $\text{SiO$_2$}$. Для отображения спектров пропускания двух материалов запустите ячейку ниже.

In [ ]:
wavelengths = np.linspace(0.3, 1.0, num=1000)
materials = ['SiO2', 'TiO2']
plt.rcParams.update({'font.size': 14})

for material in materials:
    plt.plot(wavelengths*1000, n(material, wavelengths), label='{0}, $<n> = {1:.2f}$'.format(material, n(material, wavelengths).mean()))

plt.xlabel('$\lambda$, нм')
plt.ylabel('$n$')
plt.legend()
plt.show()

Можно видеть, что в отличие от $\text{SiO$_2$}$, показатель преломления $\text{TiO$_2$}$ нельзя считать постоянным в видимом диапазоне длин волн. Таким образом, теоретическое рассмотрение, принимающее коэффициенты пропускания постоянными величинами, являются упрощенными, и при внимательном отношении не вполне корректны. В работе мы будем учитывать дисперсию показателя преломления для каждого из материалов.

### 2.2. Исследование влияние числа оптических ячеек

Ключевой особенностью фотонных кристаллов является наличие запрещенной зоны для распространения фотонов, называемаой также стоп-зоной (англ. stop band). То есть фотоны с определенными энергиями (длинами волн) не могут распространяться перпендикулярно слоям одномерного фотонного кристала, и отражаются в обратном направлении. Значения коэффициента пропускания для области стоп-зоны близки к нулю (коэффициент отражения при этом близок к 100%). Таким образом, фотонный кристалл выступает в качестве избирательного зеркала, что эффективно используется для создания резонаторов и фильтров лазерных структур.

Однако не всякий слоистый материал обладает фотонной запрещенной зонной. Так, структура должна состоять из достаточно большого числа экивалентных оптических ячеек. Проиллюстрируем это на модельном эксперименте, постепенно увеличивая число оптических ячеек, созданных на кварцевой подложке, от 1 до 9.

In [ ]:
plt.figure(figsize=(16,12))

air_in  = ['air', 1000]
substr  = ['SiO2', 1000]
air_out = ['air', 1000]

pair = ['TiO2', 70+h[var],
        'SiO2', 70+h[var]]

for N in range(1, 10):
    plt.subplot(3, 3, N)
    structure = air_in + N*pair + substr + air_out
    T(structure, xmin=400, xmax=1000)
    plt.title('Число оптических ячеек - {}'.format(N))

print('Толщина каждого слоя в соответствии с вариантом {0} составляет {1} нм.'.format(var, 70+h[var]))    

plt.show()

Сохраните полученный результат в отчет по работе (рисунок можно сохранить через контекстное меню по клику на нем правой клавишей мыши). Сделайте вывод о минимальном числе оптических ячеек, необходимых для данных материалов, чтобы наблюдалась фотонная запрещенная зона.

### 2.3. Исследование влияния периода фотонного кристалла

Итак, чтобы периодическая структура начинала существенным образом влиять на распространение электромагнитного излучения, необходимо некоторое количество оптических ячеек. Очевидно, что период такой структуры влияет на ее спектральные свойства. Исследуем это влияние. Проварьируем толщину составляющих оптическую ячейку слоев (число оптических ячеек равно семи). 

In [ ]:
plt.figure(figsize=(16,12))

air_in  = ['air', 1000]
substr  = ['SiO2', 1000]
air_out = ['air', 1000]

for N in range(1, 10):
    plt.subplot(3, 3, N)
    structure = air_in + 7*['TiO2', N*10+30+h[var], 'SiO2', N*10+30+h[var]] + substr + air_out
    T(structure, xmin=400, xmax=1000)
    plt.title('Толщины слоев SiO$_2$ и TiO$_2$ - {} нм'.format(N*10+30+h[var]))

plt.show()

В результате эксперимента наблюдается постепенное смещение и расширение интервала наблюдавшейся ранее стоп-зоны, а также проявление дополнительной стоп-зоны. То есть аналогично зонной теории электронов в твердом теле, здесь мы наблюдаем чередование разрешенных и запрещенных зон. Однако, в отличие от электронов, фотоны имеют целое спиновое число (ноль), и описываются не статистикой Ферми-Дирака, а статистикой Бозе-Эйнштейна.

Сохраните полученный результат в отчет по работе. Сделайте вывод о влиянии толщины слоев оптической ячейки на положение стоп-зоны.

### 2.4. Исследование влияния оптического контраста

Разницу показателей преломления $\Delta n$ называют *оптическим контрастом*. При увелечении оптического контраста происходит увеличении ширины фотонной запрещенной зоны, а сама стоп-зона проявляется при меньшем числе оптических ячеек. Следующий эксперимент демонстирует, почему не любая слоистая периодическая структура является фотонным кристаллом.

Так как для рассматриваемых материалом оптический контраст фиксирован, в данном численном эксперименте используются модельные материалы с разными показателями преломления. Толщина обоих материалов составляет 100 нм.

In [ ]:
plt.figure(figsize=(16,12))

air_in  = ['air', 1000]
air_out = ['air', 1000]

indices['n1'] = '1*x/x'

for N in range(1, 10):
    plt.subplot(3, 3, N)
    n2 = '(1+0.2*{})*x/x'.format(N)
    indices['n2'] = n2
    structure = air_in + 7*['n1', 100, 'n2', 100] + air_out
    T(structure, xmin=200, xmax=1200)
    plt.title('$\Delta n = {:.1f}$'.format(0.2*N))

plt.show()

Сохраните полученный результат в отчет по работе. Сделайте вывод о влиянии оптического контраста на спектр пропускания периодической слоистой структуры.

### 2.5. Исследование условий возникновения состояний дефектов

Нарушение периодической структуры кристаллической решетки, создаваемое атомом примеси или дефектом, приводит к возникновению электронных состояний, расположенных внутри запрещенной зоны. Аналогичным образом изменение толщины одного из слоев 

В расчетном эксперименте, представленном ниже в центр структуры из 8 оптических ячеек добавляется слой диоксида титана с толщиной, варьирующейся от 0 нм (отсутствие дефекта) до полной толщины слоя диоксида титана. При этом в стоп-зоне образуется узкий энергетический зазор, в котором разрешено распространением фотонов с определенной энергией.

Обратите внимание на сдвиг энергетического уровня и перераспределения коэффициента пропускания вблизи края стоп-зоны. 

In [ ]:
plt.figure(figsize=(16,12))

air_in  = ['air', 1000]
substr  = ['SiO2', 1000]
air_out = ['air', 1000]

pair = ['TiO2', 70+h[var],
        'SiO2', 70+h[var]]

for N in range(1, 10):
    plt.subplot(3, 3, N)
    h_def = (70+h[var])*0.125*(N-1)
    structure = air_in + 4*pair + ['TiO2', h_def] +4*pair + substr + air_out
    T(structure, xmin=400, xmax=1000)
    plt.title('Толщина дефектного слоя - {0} нм'.format(int(h_def)))

plt.show()

print('Толщина остальных слоев фотонного кристалла составляет {0} нм (соответствует варианту {1}).'.format(70+h[var], var))

Сохраните полученный результат в отчет по работе. Сделайте вывод о влиянии на оптический спектр дефектов в структуре одномерного фотонного кристалла.

## 3. Исследование спектров пропускания реальных фотонных кристаллов

Измерьте спектры поглощения образцов фотонных кристаллов, для чего последовательно выполните следующие действия:
1. По окончании прогрева спектрофотометра запустить программу измерения спектров SC5400.
2. Задать диапазон и шаг сканирования, нажав кнопку «Параметры сканирования» (при подведении курсора к кнопкам программа отображает соответствующие подсказки). В открывшемся окне в графах поля «Длина волны» задать  по указанию преподавателя шаг измерений, начальную и конечную длины волн.
3. Проверить, что в держателе образцов не находится образца. Нажать кнопку «Сканировать нулевую линию». В данном случае образцом сравнения является воздух.
4. По окончании сканирования нулевой линии открыть отделение для образцов, закрепить исследуемый образец в держателе спектрофотометра, закрыть крышку.
5. Перемещая ручку держателя образцов, поставить образец в позицию для измерений и сканировать его спектр, нажав кнопку «Сканировать образец». По окончании измерения спектра нажать «Файл», далее «Экспорт» и экспортировать образец в xls-файл на сменный носитель под названием.
6. Записать в протокол материал обрацов фотонных кристаллов и ориентировочное положение стоп-зоны. 
7.  Повторить измерения для остальных образцов.
8. Сохранить спектры на личном носителе информации.

## 4. Моделирование спектров реальных фотонных кристаллов

Пользуясь результатами моделирования и экспериментального измерения спектров, подберите для каждого фотонного кристалла структуру, в которой будет наблюдаться максимально близкие по величине положение стоп-зоны и среднее значения коэффициента пропускания в стоп-зоне. Для этого в ячейке ниже проварьируйте описание структуры. Шаблон описания структуры состоит из чередующихся строк, каждая из которых содержит название материала и его толщину в нм. Элементы разделены запятыми. Вы можете удалять и добавлять строки, менять порядок и толщину материалов фотонного кристалла.

Обратите внимание: Jupyter поддерживает установку множественных курсоров. Щелкнув в одной части ячейки и зажав клавишу [Ctrl], вы можете  создать несколько курсоров, после чего можно в этих позициях одновременно вводить/удалять символы. Щелчок на другом месте ячейки оставляет один курсор. 

In [ ]:
plt.figure(figsize=(8,6))

# структура: материалы и толщины [нм]
structure = ['air', 1000,
             'SiO2', 1000,
             'TiO2', 70,
             'SiO2', 70,
             'TiO2', 70,
             'SiO2', 70,
             'TiO2', 70,
             'SiO2', 70,
             'TiO2', 70,
             'SiO2', 70,
             'TiO2', 70,
             'SiO2', 70,
             'TiO2', 70,
             'SiO2', 70,
             'TiO2', 70,
             'SiO2', 70,
             'TiO2', 70,
             'SiO2', 70,
             'TiO2', 70,
             'SiO2', 70,
             'air', 1000]

T(structure, xmin=400, xmax=1000)

plt.show()

Подобранные толщины слоев $h_\text{SiO2}$ и $h_\text{TiO2}$ и количество оптических ячеек $N$ занесите в таблицу отчета по приведенному ниже шаблону.

| Образец | Интервал длин волн стоп-зоны, нм | $h_\text{SiO2}$, нм | $h_\text{TiO2}$, нм | $N$ |
|---------|----------------------------------|---------------------|---------------------|-----|
| ФК1     |                                  |                     |                     |     |
| ФК2     |                                  |                     |                     |     |
| ФК3     |                                  |                     |                     |     |

## Обработка результатов

1. Приведите результаты моделирования структур фотонных кристаллов. Сделайте выводы по каждой части задания.
2. Постройте экспериментально полученные спектры пропускания в координатах $T(\lambda)$. Сделайте выводы о расположении стоп-зон исследованных ФК.
3. Соотнеся результаты моделирования с результатами измерения спектров, сделайте предположения о структуре исследованных фотонных кристаллов относительно следующих параметров: а) период фотонного кристалла, б) число оптических ячеек. Результаты представьте в виде таблицы.
4. Опишите в выводах возможные причины технологические отклонений, вызывающие отличие получаемых спектров реальных фотонных кристаллов от модельного спектра.

## Контрольные вопросы

1. Какой параметр/параметры определяют положение стоп-зоны относительно шкалы длин волн фотонного кристалла?
2. Какой параметр/параметры определяют ширину стоп-зоны фотонного кристалла?
3. Какой параметр/параметры определяют величину пропускания в области стоп-зоны фотонного кристалла?
4. Каким образом можно использовать описанные эффекты на практике?

## Литература

Белотелов В.И., Звездин А.К. Фотонные кристаллы и другие метаматериалы. М.:Бюро Квантум, 2006.